Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [85]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [86]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [87]:
filename = "text8.zip"

def read_data(filename):
  with zipfile.ZipFile(filename) as f:
    name = f.namelist()[0]
    data = tf.compat.as_str(f.read(name))
  return data
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [88]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

99999000 ons anarchists advocate social relations based upon voluntary as
1000  anarchism originated as a term of abuse first used against earl


Utility functions to map characters to vocabulary IDs and back.

In [89]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


Function to generate a training batch for the LSTM model.

In [90]:
class BatchGenerator(object):
    def __init__(self, text, size, unrolls):
        self._text = text
        self._text_size = len(text)
        self._batch_size = size
        self._unrolls = unrolls
        segment = self._text_size // size
        self._cursor = [ offset * segment for offset in range(size)]
        self._last_batch = self._next_batch()
  
    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch
  
    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by unrolls new ones.
        """
        batches = [self._last_batch]
        for step in range(self._unrolls):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches

def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, 5, 10)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(train_text[:30])

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

ons anarchists advocate social
['ons anarchi', 'y eight inc', 'o two one n', 'ard roberts', 'cilities in']
['ists advoca', 'ch two zero', 'nine zero t', 's joseph wh', 'ncluding ku']
[' a']
['an']


In [91]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def make_one_hot(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_single_output():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [92]:
def construct_verbose(node_count, batch_size, unrolls):
    graph = tf.Graph()
    with graph.as_default():
        # Parameters:
        # Input gate: input, previous output, and bias.
        ix = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        im = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        ib = tf.Variable(tf.zeros([1, node_count]))
        # Forget gate: input, previous output, and bias.
        fx = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        fm = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        fb = tf.Variable(tf.zeros([1, node_count]))
        # Memory cell: input, state and bias.                             
        cx = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        cm = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        cb = tf.Variable(tf.zeros([1, node_count]))
        # Output gate: input, previous output, and bias.
        ox = tf.Variable(tf.truncated_normal([vocabulary_size, node_count], -0.1, 0.1))
        om = tf.Variable(tf.truncated_normal([node_count, node_count], -0.1, 0.1))
        ob = tf.Variable(tf.zeros([1, node_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, vocabulary_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([vocabulary_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
            forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
            update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        for _ in range(unrolls + 1):
            train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
        train_inputs = train_data[:unrolls]
        train_labels = train_data[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    return {
        "graph": graph,
        "batch_size": batch_size,
        "unrolls": unrolls,
        "train_data": train_data,
        "optimizer": optimizer,
        "loss": loss,
        "train_prediction": train_prediction,
        "learning_rate": learning_rate,
        "reset_sample_state": reset_sample_state,
        "sample_input": sample_input,
        "sample_prediction": sample_prediction,
        "output_to_input": make_one_hot,
        "random_output": random_single_output
    }

In [93]:

def run_lstm(setup, batcher, training, validation, step_count, report_every):
    train_batches = batcher(training, setup["batch_size"], setup["unrolls"])
    valid_batches = batcher(validation, 1, 1)
    with tf.Session(graph=setup["graph"]) as session:
        tf.initialize_all_variables().run()
        print('Initialized')
        mean_loss = 0
        for step in range(step_count + 1):
            batches = train_batches.next()
            feed_dict = dict()
            for i in range(setup["unrolls"] + 1):
                feed_dict[setup["train_data"][i]] = batches[i]

            _, l, predictions, lr = session.run(
                [setup["optimizer"], setup["loss"], setup["train_prediction"], setup["learning_rate"]],
                feed_dict=feed_dict
            )

            mean_loss += l
            if step % report_every == 0:
                if step > 0:
                    mean_loss = mean_loss / report_every
                # The mean loss is an estimate of the loss over the last few batches.
                print('Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
                mean_loss = 0
                labels = np.concatenate(list(batches)[1:])
                print(predictions.shape)
                print(labels)
                print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
                if step % (report_every * 10) == 0:
                    # Generate some samples.
                    print('=' * 80)
                    for _ in range(5):
                        feed = setup["output_to_input"](setup["random_output"]())
                        sentence = characters(feed)[0]
                        setup["reset_sample_state"].run()
                        for _ in range(79):
                            prediction = setup["sample_prediction"].eval({setup["sample_input"]: feed})
                            feed = setup["output_to_input"](prediction)
                            sentence += characters(feed)[0]
                        print(sentence)
                    print('=' * 80)
                # Measure validation set perplexity.
                setup["reset_sample_state"].run()
                valid_logprob = 0
                for _ in range(len(validation)):
                    b = valid_batches.next()
                    predictions = setup["sample_prediction"].eval({setup["sample_input"]: b[0]})
                    valid_logprob = valid_logprob + logprob(predictions, b[1])
                print('Validation set perplexity: %.2f' % float(np.exp(valid_logprob / valid_size)))

In [94]:
verbose_setup = construct_verbose(node_count=64, batch_size=64, unrolls=10)

run_lstm(verbose_setup, BatchGenerator, train_text, valid_text, step_count=7000, report_every=100)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.292521 learning rate: 10.000000
(640, 27)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Minibatch perplexity: 26.91
lt rin potw fx gp ziiqtcynhs  kt n wirpt qbtrra efw tubl  l  t do izinap xkiiftb
rlnasxblmbiycnjunegn nbjafdnimi  gb  zsbriirjwxodhe  mg il aiwy feciantfamneyul 
ygm   rsol cxiiazosx wirrgfsdwvdteesx tcr rrzfotadokpsjqtyp ayrxhqtdgikok g t so
 ali s lnroqrco cyedtpl  lipq oebozsee  hoenarhudtuisiccnbsresjg rvrytfdz bmpncw
vymcyhnppdrcmdf tsljtowbsshiedeoj o iu  tgsyuopnu am nmvhy   sdy lbaxswee qabsih
Validation set perplexity: 19.89
Average loss at step 100: 2.590023 learning rate: 10.000000
(640, 27)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [95]:
def construct_compact(node_count, batch_size, unrolls):
    graph = tf.Graph()
    with graph.as_default():
        gate_count = 4
        # Parameters:
        # Gates: input, previous output, and bias.
        input_weights = tf.Variable(tf.truncated_normal([vocabulary_size, node_count * gate_count], -0.1, 0.1))
        output_weights = tf.Variable(tf.truncated_normal([node_count, node_count * gate_count], -0.1, 0.1))
        bias = tf.Variable(tf.zeros([1, node_count * gate_count]))
        # Variables saving state across unrollings.
        saved_output = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        saved_state = tf.Variable(tf.zeros([batch_size, node_count]), trainable=False)
        # Classifier weights and biases.
        w = tf.Variable(tf.truncated_normal([node_count, vocabulary_size], -0.1, 0.1))
        b = tf.Variable(tf.zeros([vocabulary_size]))

        # Definition of the cell computation.
        def lstm_cell(i, o, state):
            """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
            Note that in this formulation, we omit the various connections between the
            previous state and the gates."""
            values = tf.split(1, gate_count, tf.matmul(i, input_weights) + tf.matmul(o, output_weights) + bias)
            input_gate = tf.sigmoid(values[0])
            forget_gate = tf.sigmoid(values[1])
            update = values[2]
            state = forget_gate * state + input_gate * tf.tanh(update)
            output_gate = tf.sigmoid(values[3])
            return output_gate * tf.tanh(state), state

        # Input data.
        train_data = list()
        for _ in range(unrolls + 1):
            train_data.append(tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
        train_inputs = train_data[:unrolls]
        train_labels = train_data[1:]  # labels are inputs shifted by one time step.

        # Unrolled LSTM loop.
        outputs = list()
        output = saved_output
        state = saved_state
        for i in train_inputs:
            output, state = lstm_cell(i, output, state)
            outputs.append(output)

        # State saving across unrollings.
        with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
            # Classifier.
            logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))

        # Optimizer.
        global_step = tf.Variable(0)
        learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        gradients, v = zip(*optimizer.compute_gradients(loss))
        gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
        optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

        # Predictions.
        train_prediction = tf.nn.softmax(logits)

        # Sampling and validation eval: batch 1, no unrolling.
        sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
        saved_sample_output = tf.Variable(tf.zeros([1, node_count]))
        saved_sample_state = tf.Variable(tf.zeros([1, node_count]))
        reset_sample_state = tf.group(
            saved_sample_output.assign(tf.zeros([1, node_count])),
            saved_sample_state.assign(tf.zeros([1, node_count]))
        )
        sample_output, sample_state = lstm_cell(sample_input, saved_sample_output, saved_sample_state)
        with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                      saved_sample_state.assign(sample_state)]):
            sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    
    return {
        "graph": graph,
        "batch_size": batch_size,
        "unrolls": unrolls,
        "train_data": train_data,
        "optimizer": optimizer,
        "loss": loss,
        "train_prediction": train_prediction,
        "learning_rate": learning_rate,
        "reset_sample_state": reset_sample_state,
        "sample_input": sample_input,
        "sample_prediction": sample_prediction,
        "output_to_input": make_one_hot,
        "random_output": random_single_output
    }

In [96]:
compact_setup = construct_compact(node_count=64, batch_size=64, unrolls=10)

run_lstm(compact_setup, BatchGenerator, train_text, valid_text, step_count=7000, report_every=100)


Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 3.296253 learning rate: 10.000000
(640, 27)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
Minibatch perplexity: 27.01
lfnfewalkvsoekr e eg jeset lj ty n t rn uizplwxiizyxeazkmucollas cu  nto s pe ci
txrtlhsxo ewzr k dnqdk pn a anigcd e nut tbos oed  aueoellatnibdnsyo vvnd pjnhud
pdttfor  cj gbvogvfsei f  loidfi  p aifa e murzrpewg fgnbrspmejpahhoitdgttjswa w
vaaltvoevkye on   xnvtj koczambntwhwyfiat esroa  usiith wrm ek qekzfl rizchmacx 
gspoaqi mkx ttnu i z bmslaecnhbddil int p twvodamni tfdsyngeubof zvlbvlbrvlebeoe
Validation set perplexity: 19.92
Average loss at step 100: 2.598242 learning rate: 10.000000
(640, 27)
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [97]:
train_batches = BatchGenerator(train_text, 5, 10)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(train_text[:30])

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

ons anarchists advocate social
['ons anarchi', 'y eight inc', 'o two one n', 'ard roberts', 'cilities in']
['ists advoca', 'ch two zero', 'nine zero t', 's joseph wh', 'ncluding ku']
[' a']
['an']


In [98]:
batch_size=64
num_unrollings=10

class BatchGeneratorBigram(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size / batch_size
    # list of offsets within batch
    self._cursor = [ offset * segment for offset in xrange(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size), dtype=np.int)  # id of char to be embedded
    for b in xrange(self._batch_size):
      batch[b] = char2id(self._text[self._cursor[b]]) # get id of a char
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size  # move cursor
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in xrange(self._num_unrollings):
      batches.append(self._next_batch())  # add id of char for 1 to num_unrollings
    self._last_batch = batches[-1]
    return batches

def bigrambatches2string(batches):
  """Convert a sequence of batches back into string
  representation.
  """
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, [id2char(c) for c in b])]
  return s

In [99]:
train_batches = BatchGeneratorBigram(train_text, batch_size, num_unrollings)
valid_batches = BatchGeneratorBigram(valid_text, 1, 1) # returns batch size 1, +1 unrolling
train_labels = BatchGenerator(train_text, batch_size, num_unrollings)
valid_labels = BatchGenerator(valid_text, 1, 1) # returns batch size 1, +1 unrolling 


print(bigrambatches2string(train_batches.next()))
print(bigrambatches2string(train_batches.next()))
print(bigrambatches2string(valid_batches.next()))
print(bigrambatches2string(valid_batches.next()))

['ons anarchi', 'when milita', 'lleria arch', ' abbeys and', 'married urr', 'hel and ric', 'y and litur', 'ay opened f', 'tion from t', 'migration t', 'new york ot', 'he boeing s', 'e listed wi', 'eber has pr', 'o be made t', 'yer who rec', 'ore signifi', 'a fierce cr', ' two six ei', 'aristotle s', 'ity can be ', ' and intrac', 'tion of the', 'dy to pass ', 'f certain d', 'at it will ', 'e convince ', 'ent told hi', 'ampaign and', 'rver side s', 'ious texts ', 'o capitaliz', 'a duplicate', 'gh ann es d', 'ine january', 'ross zero t', 'cal theorie', 'ast instanc', ' dimensiona', 'most holy m', 't s support', 'u is still ', 'e oscillati', 'o eight sub', 'of italy la', 's the tower', 'klahoma pre', 'erprise lin', 'ws becomes ', 'et in a naz', 'the fabian ', 'etchy to re', ' sharman ne', 'ised empero', 'ting in pol', 'd neo latin', 'th risky ri', 'encyclopedi', 'fense the a', 'duating fro', 'treet grid ', 'ations more', 'appeal of d', 'si have mad']
['ists advoca', 'ary governm', 'hes nat

In [100]:
num_nodes = 64
#vocabulary_size = (len(string.ascii_lowercase) + 1)**2
embedding_size = 128 # Dimension of the embedding vector.
batch_size=64
num_unrollings=10



graph = tf.Graph()
with graph.as_default():
  
  ## Parameters:
  fico_x = tf.Variable(tf.truncated_normal([4, embedding_size, num_nodes], -0.1, 0.1))
  print(fico_x.get_shape().as_list())
  fico_m = tf.Variable(tf.truncated_normal([4, num_nodes, num_nodes], -0.1, 0.1))
  fico_b = tf.Variable(tf.zeros([4, 1, num_nodes]))
    
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)

  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
    
  # Embedding Variables.
  embeddings = tf.Variable(
    tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), trainable=False)
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """
    Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates.
    """                   
    i_list = tf.pack([i, i, i, i])
    o_list = tf.pack([o, o, o, o])
                          
    ins = tf.batch_matmul(i_list, fico_x)
    outs = tf.batch_matmul(o_list, fico_m)
    
    h_x = ins + outs + fico_b

    forget_gate = tf.sigmoid(h_x[0,:,:])

    input_gate = tf.sigmoid(h_x[1,:,:])
    update = tf.tanh(h_x[2,:,:])
    state = forget_gate*state + input_gate*update
    
    output_gate = tf.sigmoid(h_x[3,:,:])
    
    h = output_gate * tf.tanh(state)
    return h, state

  # Input data.
  train_data = list()
  train_data_y = list()
  for _ in xrange(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.int32, shape=[batch_size]))  # removed ohe of char
    train_data_y.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))  # uses ohe of char
  train_labels = train_data_y[1:]
  
  # Embedded input data
  encoded_inputs = list()
  for bigram in train_data:
    embed = tf.nn.embedding_lookup(embeddings, bigram)
    encoded_inputs.append(embed)
  train_inputs = encoded_inputs[:num_unrollings]

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
    print('logits', logits.get_shape().as_list())
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)))
    print('labels', tf.concat(0, train_labels).get_shape().as_list())

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=False)  ## orig 10.0, 5000, 0.1, True
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.int32, shape=[1]) # removed ohe of char
  sample_input_emb = tf.nn.embedding_lookup(embeddings, sample_input)
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input_emb, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))


[4, 128, 64]
logits [640, 27]
labels [640, 27]


In [101]:
import time
# training and validation batches
train_batches = BatchGeneratorBigram(train_text, batch_size, num_unrollings)
valid_batches = BatchGeneratorBigram(valid_text, 1, 1) # returns batch size 1, +1 unrolling
train_batches_y = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches_y = BatchGenerator(valid_text, 1, 1) # returns batch size 1, +1 unrolling 

num_steps = 7001  ## orig 7001
summary_frequency = 100

t0 = time.time()
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized\n==========')
  mean_loss = 0
  for step in xrange(num_steps):
    batches = train_batches.next()
    batches_y = train_batches_y.next()
    
    feed_dict = dict()
    for i in xrange(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
      feed_dict[train_data_y[i]] = batches_y[i]
    
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % (5.*summary_frequency) == 0:  ## orig 2.5*summary_frequency
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print('Average loss at step', step, '=', mean_loss, '\nlearning rate:', lr)
      mean_loss = 0
      labels = np.concatenate(list(batches_y)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in xrange(5):
          #feed = sample(random_distribution())  # random vector
          feed = np.random.randint(27, size=[1])#.astype('int32')
          #sentence = characters(feed)[0]
          sentence = id2char(feed)
          reset_sample_state.run()
          for _ in xrange(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)  # get ohe of predicted proba
            feed = np.array([np.argmax(feed)])  # get id of predicted char
            sentence += id2char(feed)  # add predicted char
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in xrange(valid_size):
        b = valid_batches.next()
        b_y = valid_batches_y.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b_y[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))
      print('-' * 30)
# show how much time elapsed
print((time.time()-t0)/60., 'minutes elapsed')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0 = 3.30546188354 
learning rate: 10.0
Minibatch perplexity: 27.26
uk a tlnd rlf pgnnrntuceryvlsbb twgximhdixbsnnsa otayftdwe vsea te mjxrsluaaeo e
ujjmnkenpebzla ceg ahpo o nnfss zabygdwavr s tiadhrbnso a  mtaooeheydno soshgbss
t n ktawpiau rbla t nvtexbrwpagrtsnwm nycrqwdu u anfxzrjvlnhy v vy  a j  cnzktan
qrbe  sfhmrk  l   mepi abo jatf wlf rquioiohl d mwsxmgnye qwmbvcxgwrvbrgiprm t o
kosjmhhsjwo  itnpgwvbfgczhgiefadhi o dtemlhs wcvrhicf cbqhcxbtthnrczjpdeaxo xhrl
Validation set perplexity: 18.79
------------------------------
Average loss at step 500 = 9.92722220778 
learning rate: 7.94328
Minibatch perplexity: 6.40
Validation set perplexity: 5.94
------------------------------
Average loss at step 1000 = 8.84984380364 
learning rate: 6.30957
Minibatch perplexity: 5.91
i lesitation bhat statc the mearge one one two zero th ethre two monian kazas a 
legat maritality to kulctan 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---

In [126]:
from __future__ import print_function
import numpy as np
import random
import string
import tensorflow as tf
import sys
from six.moves import range

In [127]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
1 26 0 0
a z  


In [138]:
import sys
sys.path.append(tf.__path__[0] + '/models')
print(sys.path)

['', '/Users/alex/anaconda2/lib/python27.zip', '/Users/alex/anaconda2/lib/python2.7', '/Users/alex/anaconda2/lib/python2.7/plat-darwin', '/Users/alex/anaconda2/lib/python2.7/plat-mac', '/Users/alex/anaconda2/lib/python2.7/plat-mac/lib-scriptpackages', '/Users/alex/anaconda2/lib/python2.7/lib-tk', '/Users/alex/anaconda2/lib/python2.7/lib-old', '/Users/alex/anaconda2/lib/python2.7/lib-dynload', '/Users/alex/anaconda2/lib/python2.7/site-packages', '/Users/alex/anaconda2/lib/python2.7/site-packages/Sphinx-1.4.6-py2.7.egg', '/Users/alex/anaconda2/lib/python2.7/site-packages/aeosa', '/Users/alex/anaconda2/lib/python2.7/site-packages/setuptools-27.2.0-py2.7.egg', '/Users/alex/anaconda2/lib/python2.7/site-packages/IPython/extensions', '/Users/alex/.ipython', '/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models', '/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models', '/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models', '/Users/alex/anaconda2/lib

In [139]:
import tensorflow.models.rnn.translate.seq2seq_model as seq2seq_model

In [130]:
text = "the quick brown fox jumps over the lazy dog is an english sentence that can be translated to the following french one le vif renard brun saute par dessus le chien paresseux here is an extremely long french word anticonstitutionnellement"

def longest_word_size(text):
    return max(map(len, text.split()))

word_size = longest_word_size(text)
print(word_size)

25


In [131]:

import string

num_nodes = 64
batch_size = 10

def create_model():
     return seq2seq_model.Seq2SeqModel(source_vocab_size=vocabulary_size,
                                   target_vocab_size=vocabulary_size,
                                   buckets=[(word_size + 1, word_size + 2)], # only 1 bucket
                                   size=num_nodes,
                                   num_layers=3,
                                   max_gradient_norm=5.0,
                                   batch_size=batch_size,
                                   learning_rate=0.5,
                                   learning_rate_decay_factor=0.99,
                                   use_lstm=True,
                                   forward_only=False)

In [132]:
def get_batch():
    encoder_inputs = [np.random.randint(1, vocabulary_size, word_size + 1) for _ in xrange(batch_size)]
    decoder_inputs = [np.zeros(word_size + 2, dtype=np.int32) for _ in xrange(batch_size)]
    weights = [np.ones(word_size + 2, dtype=np.float32) for _ in xrange(batch_size)]
    for i in xrange(batch_size):
        r = random.randint(1, word_size)
        # leave at least a 0 at the end
        encoder_inputs[i][r:] = 0
        # one 0 at the beginning of the reversed word, one 0 at the end
        decoder_inputs[i][1:r+1] = encoder_inputs[i][:r][::-1]
        weights[i][r+1:] = 0.0
    return np.transpose(encoder_inputs), np.transpose(decoder_inputs), np.transpose(weights)

In [133]:
def strip_zeros(word):
    # 0 is the code for space in char2id()
    return word.strip(' ')

def evaluate_model(model, sess, words, encoder_inputs):
    correct = 0
    decoder_inputs = np.zeros((word_size + 2, batch_size), dtype=np.int32)
    target_weights = np.zeros((word_size + 2, batch_size), dtype=np.float32)
    target_weights[0,:] = 1.0
    is_finished = np.full(batch_size, False, dtype=np.bool_)
    for i in xrange(word_size + 1):
        _, _, output_logits = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_id=0, forward_only=True)
        p = np.argmax(output_logits[i], axis=1)
        is_finished = np.logical_or(is_finished, p == 0)
        decoder_inputs[i,:] = (1 - is_finished) * p
        target_weights[i,:] = (1.0 - is_finished) * 1.0
        #if np.all(is_finished):
            #break
    print(decoder_inputs)
    for idx, l in enumerate(np.transpose(decoder_inputs)):
        reversed_word = ''.join(reversed(words[idx]))
        output_word = strip_zeros(''.join(id2char(i) for i in l))
        print(words[idx], '(reversed: {0})'.format(reversed_word),
              '->', output_word, '({0})'.format('OK' if reversed_word == output_word else 'KO'))
        if reversed_word == output_word:
            correct += 1
    return correct

In [134]:
def get_validation_batch(words):
    encoder_inputs = [np.zeros(word_size + 1, dtype=np.int32) for _ in xrange(batch_size)]
    for i, word in enumerate(words):
        for j, c in enumerate(word):
            encoder_inputs[i][j] = char2id(c)
    return np.transpose(encoder_inputs)

def validate_model(text, model, sess):
    words = text.split()
    nb_words = (len(words) / batch_size) * batch_size
    correct = 0
    for i in xrange(nb_words / batch_size):
        range_words = words[i * batch_size:(i + 1) * batch_size]
        encoder_inputs = get_validation_batch(range_words)
        correct += evaluate_model(model, sess, range_words, encoder_inputs)
    print('* correct: {0}/{1} -> {2}%'.format(correct, nb_words, (float(correct) / nb_words) * 100))
    print()

In [135]:
def reverse_text(nb_steps):
    with tf.Session() as session:
        model = create_model()
        tf.initialize_all_variables().run()
        for step in xrange(nb_steps):
            enc_inputs, dec_inputs, weights = get_batch()
            _, loss, _ = model.step(session, enc_inputs, dec_inputs, weights, 0, False)
            if step % 1000 == 1:
                print('* step:', step, 'loss:', loss)
                validate_model(text, model, session)
        print('*** evaluation! loss:', loss)
        validate_model(text, model, session)

In [140]:
%time reverse_text(15000)

ValueError: Variable embedding_attention_seq2seq/RNN/EmbeddingWrapper/embedding already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models/rnn/translate/seq2seq_model.py", line 135, in seq2seq_f
    dtype=dtype)
  File "/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models/rnn/translate/seq2seq_model.py", line 171, in <lambda>
    lambda x, y: seq2seq_f(x, y, False),
  File "/Users/alex/anaconda2/lib/python2.7/site-packages/tensorflow/models/rnn/translate/seq2seq_model.py", line 172, in __init__
    softmax_loss_function=softmax_loss_function)
